In [1]:
import csv
from lxml import html
import pandas as pd
import numpy as np
import nltk
import string
import re
from collections import defaultdict
from nltk.stem.snowball import EnglishStemmer
from pickle import load

# Loading stopwords

In [2]:
nltk.download('stopwords')
tokenizer = nltk.RegexpTokenizer(r'\w+')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mlmaster/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Loading Data

In [3]:
x = pd.read_csv('QueryResults.csv')[0:10]
title = x['Title']
body = x['Body']

In [4]:
title

0              Macro only running through first result
1         Set limitation on gap tolerance and run time
2    Connect two different Database with one applic...
3    bootstrap 4 navbar disappears when resizing sc...
4                   xml transform XSLT with namespaces
5    Convert Timestamp to Date with various formats...
6    What is the best practice for estimating requi...
7                         How to find an object by id?
8    Deep learning: parameter selection and result ...
9              Connect multiple databases with laravel
Name: Title, dtype: object

In [5]:
to_keep_ = open("Flask/to_keep.csv", 'r')
to_keep_ = csv.reader(to_keep_)
to_keep = list(to_keep_)
to_keep = [item for sublist in to_keep for item in sublist]

sw = open("Flask/sw.csv", 'r')
sw = csv.reader(sw)
sw = list(sw)
sw = [item for sublist in sw for item in sublist]

# Building the pipeline

In [6]:
def pipe(title, body):
    body = title + ' ' + body
    body = title + ' ' + body
    body = html.fromstring(body).text_content()
    body = body.replace('\n', '')

    # There is still some html tags, let's remove them with a regex
    TAG_RE = re.compile(r'<[^>]+>')
    body = TAG_RE.sub('', body)
    # Init stemmer and tokenizer
    stemmer = EnglishStemmer()
    tokenizer = nltk.RegexpTokenizer(r'\w+')
    # Remove stop words and keep only the good words
    token = tokenizer.tokenize(body.lower())
    desc = [stemmer.stem(w) for w in token if not w in sw and w in to_keep]
    # Transforming the result to a single row
    desc = pd.DataFrame(desc).T.apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
    
    return desc

In [7]:
x.iloc[4,:]

Id                                                                48320558
PostTypeId                                                               1
AcceptedAnswerId                                                  48321030
ParentId                                                               NaN
CreationDate                                           2018-01-18 11:44:57
DeletionDate                                                           NaN
Score                                                                    0
ViewCount                                                              196
Body                     <p>I have a xml generated from SharePoint and ...
OwnerUserId                                                         777551
OwnerDisplayName                                                       NaN
LastEditorUserId                                                       NaN
LastEditorDisplayName                                                  NaN
LastEditDate             

In [8]:
tfidf = load(open('Flask/tfidf', 'rb'))
model = load(open('Flask/model', 'rb'))
mb = load(open('Flask/mb', 'rb'))

In [26]:
pip = pipe(title[6], body[6])
cleaned = tfidf.transform(pip)
result = model.predict_proba(cleaned)

In [32]:
res = mb.inverse_transform(model.predict(cleaned))[0]
print(res)
res = [i for i in res if i != '']
res

('',)


[]

In [35]:
if res:
    print('ok')
    
else:
    print('null')

null


In [11]:
result#[0] >= 0.3

array([[8.76470549e-01, 8.44779388e-03, 4.60697874e-04, 7.48541961e-04,
        6.03596519e-04, 1.86885253e-03, 7.00424366e-04, 9.88116932e-03,
        6.20164630e-04, 5.50807138e-04, 3.04810668e-03, 4.38570307e-04,
        7.15535662e-04, 1.92586091e-03, 4.32941329e-04, 5.49348687e-04,
        1.84918692e-03, 3.60980647e-03, 6.56268523e-04, 4.27580456e-04,
        4.91146832e-04, 8.28485480e-04, 1.39794628e-03, 2.69034133e-04,
        7.84723551e-04, 3.91230198e-04, 5.99581260e-04, 6.01358915e-04,
        7.83104776e-04, 1.28485827e-03, 8.53111391e-04, 1.11328020e-03,
        5.33312305e-04, 4.11624822e-04, 2.67034987e-03, 4.36086302e-04,
        9.31683088e-04, 5.26079035e-04, 4.31410761e-04, 6.45236586e-04,
        1.08758236e-03, 9.98704882e-02, 2.17066936e-03, 2.93010841e-04,
        6.20022183e-04, 7.29622597e-04, 6.95305210e-04, 7.38435741e-04,
        7.49011429e-04, 5.62054885e-04, 9.98395674e-04, 6.28094511e-04,
        1.48775310e-03, 5.57782914e-03, 5.85664418e-04, 8.446614

In [12]:
mb.classes_

array(['', '.net', '.net-core', 'aggregate', 'ajax', 'algorithm',
       'amazon-web-services', 'android', 'android-layout',
       'android-studio', 'angular', 'angularjs', 'animation', 'apache',
       'apache-spark', 'apache-spark-sql', 'api', 'architecture',
       'arguments', 'arraylist', 'arrays', 'arrow-functions', 'asp.net',
       'asp.net-core', 'asp.net-mvc', 'asp.net-web-api', 'assembly',
       'async-await', 'asynchronous', 'audio', 'authentication', 'awk',
       'axios', 'azure', 'bash', 'batch-file', 'beautifulsoup', 'boolean',
       'bootstrap-4', 'button', 'c', 'c#', 'c++', 'c++11', 'c++14',
       'c++17', 'canvas', 'casting', 'char', 'class', 'cmake', 'cmd',
       'codeigniter', 'coding-style', 'collections', 'command-line',
       'compiler-errors', 'conditional-statements', 'constructor', 'cors',
       'count', 'css', 'css-grid', 'css-selectors', 'csv', 'curl', 'dart',
       'data-structures', 'data.table', 'database', 'database-design',
       'dataframe', 

In [13]:
mb.classes_[196]

'networking'